In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
import math

# Загрузка данных и их обработка

In [130]:
iris = datasets.load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['target'] = iris.target

data = data.loc[(data["target"] == 1) | (data["target"] == 2)] # выбор нужных классов ириса
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  100 non-null    float64
 1   sepal width (cm)   100 non-null    float64
 2   petal length (cm)  100 non-null    float64
 3   petal width (cm)   100 non-null    float64
 4   target             100 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 4.3 KB


In [131]:
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [132]:
X = data.drop(["target"], axis=1)
Y = data["target"].map({2: 1, 1: 0})
Y.value_counts()

0    50
1    50
Name: target, dtype: int64

In [133]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Реализация класса логистической регрессии

In [134]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.weight = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def cross_entropy(self, h0, y, m):
        h0 = np.clip(h0, 0.0001, 0.9999)
        return np.sum(-y * np.log(h0) - (1 - y) * np.log(1 - h0))/m

    def fit_gradient(self, X_train, Y_train, num_iterations=10000):
        X_train = X_train.values
        Y_train = Y_train.values
        X_train = X_train.T
        Y_train = Y_train.reshape(1, X_train.shape[1])
        m = X_train.shape[1]
        n = X_train.shape[0]
        self.weight = np.random.randn(1, n)
        self.bias = 0
        cost_list = []
        for i in range(num_iterations):
            z = np.dot(self.weight, X_train) + self.bias
            h0 = self.sigmoid(z)
            update_weight = np.dot(h0 - Y_train, X_train.T) / m
            update_bias = np.sum(h0 - Y_train) / m
            self.weight = self.weight - self.learning_rate * update_weight
            self.bias = self.bias - self.learning_rate * update_bias
            if i % 1000 == 0:
                cost = self.cross_entropy(h0, Y_train, m)
                cost_list.append(cost)
                print(f"После {i} итерации значение функции стоимости = {cost}")
        return cost_list

    def fit_rmsprop(self, X_train, Y_train, beta=0.9, eps=1e-8, num_iterations=1000):
        X_train = X_train.values
        Y_train = Y_train.values
        X_train = X_train.T
        Y_train = Y_train.reshape(1, X_train.shape[1])
        m = X_train.shape[1]
        n = X_train.shape[0]
        self.weight = np.random.randn(1, n)
        self.bias = 0
        v_dw = np.zeros_like(self.weight)
        v_db = 0
        cost_list = []
        for i in range(num_iterations):
            z = np.dot(self.weight, X_train) + self.bias
            h0 = self.sigmoid(z)
            dw = np.dot(h0 - Y_train, X_train.T) / m
            db = np.sum(h0 - Y_train) / m
            v_dw = beta * v_dw + (1 - beta) * dw**2
            v_db = beta * v_db + (1 - beta) * db**2
            self.weight -= self.learning_rate * dw / (np.sqrt(v_dw) + eps)
            self.bias -= self.learning_rate * db / (np.sqrt(v_db) + eps)
            if i % 100 == 0:
                cost = self.cross_entropy(h0, Y_train, m)
                cost_list.append(cost)
                print(f"После {i} итерации значение функции стоимости = {cost}")
        return cost_list
    
    def fit_nadam(self, X_train, Y_train, mu=0.975, nu=0.999, eps=1e-8, num_iterations=2000):
        X_train = X_train.values
        Y_train = Y_train.values
        X_train = X_train.T
        Y_train = Y_train.reshape(1, X_train.shape[1])
        m = X_train.shape[1]
        n = X_train.shape[0]
        self.weight = np.random.randn(1, n)
        self.bias = -7.380594851807613 # взят по методу RMSProp
        m_param = np.zeros_like(self.weight)
        n_param = np.zeros_like(self.weight)
        cost_list = []
        for i in range(num_iterations):
            z = np.dot(self.weight, X_train) + self.bias
            h0 = self.sigmoid(z)
            dw = np.dot(h0 - Y_train, X_train.T) / m
            m_param = mu * m_param + (1 - mu) * dw
            n_param = nu * n_param + (1 - nu) * dw**2
            mhat = (mu * m_param / (1 - mu)) + ((1 - mu) * dw / (1 - mu))
            nhat = nu * n_param / (1 - nu)
            self.weight -= self.learning_rate / (np.sqrt(nhat) + eps) * mhat
            if i % 100 == 0:
                cost = self.cross_entropy(h0, Y_train, m)
                cost_list.append(cost)
                print(f"После {i} итерации значение функции стоимости = {cost}")
        return cost_list
    
    def predict(self, X_pred):
        X_pred = X_pred.values
        X_pred = X_pred.T
        z = np.dot(self.weight, X_pred) + self.bias
        h0 = self.sigmoid(z)
        Y_pred = np.zeros((1, X_pred.shape[1]))
        for i in range(h0.shape[1]):
            if h0[0, i] <= 0.5:
                Y_pred[0, i] = 0
            else:
                Y_pred[0, i] = 1
        Y_pred = Y_pred.flatten()
        Y_pred = pd.DataFrame({"Result": Y_pred})
        return Y_pred

# Обучение

In [135]:
from sklearn.metrics import accuracy_score

results = []

model = MyLogisticRegression()
model.fit_gradient(X_train, Y_train)
Y_pred = model.predict(X_test)
results.append(accuracy_score(Y_pred, Y_test))

После 0 итерации значение функции стоимости = 4.282778174826798
После 1000 итерации значение функции стоимости = 0.4302604965889143
После 2000 итерации значение функции стоимости = 0.30698567612636535
После 3000 итерации значение функции стоимости = 0.2454092633655421
После 4000 итерации значение функции стоимости = 0.20849523874277515
После 5000 итерации значение функции стоимости = 0.18371028720346924
После 6000 итерации значение функции стоимости = 0.16577391249773912
После 7000 итерации значение функции стоимости = 0.15209225630170686
После 8000 итерации значение функции стоимости = 0.1412437542001003
После 9000 итерации значение функции стоимости = 0.13238338638801883


In [136]:
model2 = MyLogisticRegression()
model2.fit_rmsprop(X_train, Y_train)
Y_pred = model2.predict(X_test)
print(model2.bias)
results.append(accuracy_score(Y_pred, Y_test))

После 0 итерации значение функции стоимости = 0.7507107466606253
После 100 итерации значение функции стоимости = 0.4265997160338322
После 200 итерации значение функции стоимости = 0.2706040154392928
После 300 итерации значение функции стоимости = 0.18553083362989867
После 400 итерации значение функции стоимости = 0.1358601173820261
После 500 итерации значение функции стоимости = 0.1047127963070876
После 600 итерации значение функции стоимости = 0.08397859965551735
После 700 итерации значение функции стоимости = 0.06946857906604283
После 800 итерации значение функции стоимости = 0.05886541936363898
После 900 итерации значение функции стоимости = 0.050821744225240884
-7.359852431115496


In [137]:
model3 = MyLogisticRegression()
model3.fit_nadam(X_train, Y_train)
Y_pred = model3.predict(X_test)
results.append(accuracy_score(Y_pred, Y_test))

После 0 итерации значение функции стоимости = 4.868369913258997
После 100 итерации значение функции стоимости = 0.833618024981764
После 200 итерации значение функции стоимости = 0.4073685770956378
После 300 итерации значение функции стоимости = 0.3505449852404651
После 400 итерации значение функции стоимости = 0.3136135378698714
После 500 итерации значение функции стоимости = 0.28323909309183504
После 600 итерации значение функции стоимости = 0.25789642899166976
После 700 итерации значение функции стоимости = 0.23647703376760876
После 800 итерации значение функции стоимости = 0.21816515646160203
После 900 итерации значение функции стоимости = 0.20234725167819081
После 1000 итерации значение функции стоимости = 0.1885555588488541
После 1100 итерации значение функции стоимости = 0.176428983698802
После 1200 итерации значение функции стоимости = 0.1656853451554405
После 1300 итерации значение функции стоимости = 0.15610153077984798
После 1400 итерации значение функции стоимости = 0.147499

# Результаты

In [138]:
result_df = pd.DataFrame(data=results, columns=["Result"], index=["Gradiend_Descent", "RMSProp", "Nadam"])
result_df

,Result
Gradiend_Descent,0.9
RMSProp,0.9
Nadam,0.9


# Вывод
Каждый из методов имеет свои преимущества и недостатки. Датасет был очень маленький, поэтому методы показали +- одинаковые результаты. Я бы выбрал метод Nadam, так как он не требует такого большого количества итераций, как Градиентный спуск, при этом имеет много переменных, которые можно настроить под себя. Но если говорить о простоте, то Градиентный спуск явно выигрывает. RMSProp во всём этом уверенный среднячок.